# Long Sequence Time Series Forecasting




## Overview
Long Sequence Time Series Forecasting (LSTF) is a branch of time series forecating that is characterized by long input and output sequences. Being able to model the the behaviour of a system over long time horizons is essential to plan effectively in a variety of domains including economics, electricity consumption and transportation.  

## RNN for LSTF
Depsite the success of Recurrent Neural Networks (RNN) to model data with temporal dependencies, these approaches often fail in the case of LSTF. Specifically: 
- RNN scale poorly to long time sequences computationally because they sequentially process data at each time step. Thus, for long time sequences, forward passes through the network are extremely slow. 
- RNN perform poorly on the LSTF task becasue they predict the output sequence one step at a time in a recursive manner. Thus, an error is accumulated over consecutive time steps causing the prediction error to increase rapidly. 

<p align="center">
<img width="600" alt="Screen Shot 2021-09-28 at 5 41 20 PM" src="https://user-images.githubusercontent.com/34798787/151047180-f22c0a54-b198-4be3-85b0-6e54fe1b38db.png">  
</p>

## Transformers for LSTF
Recently, Transformer models have shown strong performance in modelling temporal dependcies when compared to RNN. This is becasue the attention mechanism reduces the max length of network signal to a theoretic $O(1)$ and avoid the reccurent structure possessed by RNN. 

<p align="center">
<img width="472" alt="Screen Shot 2022-01-25 at 2 54 40 PM" src="https://user-images.githubusercontent.com/34798787/151049619-abc40b66-91a1-43be-bee3-50232e1f5a99.png">
</p>

However, Transformers require $O(N^2)$ in both computation and memory where $N$ is the the sequence length. Thus, this bottleneck limits the use of Transformers out of the box for LSTF. Because of this, there have been several works proposed to to surmount the quadratic computation and memory consumption of Transformers by augmenting the attention mechanism. These approaches include 
[Sparse Transformer](https://arxiv.org/pdf/1904.10509v1.pdf), [LogSparse Transformer](https://arxiv.org/pdf/1904.10509v1.pdf) and [Linformer](https://arxiv.org/abs/2006.04768) among others. Specifically, for the LSTF problem, there have been a few approaches proposed including [Informer](https://arxiv.org/abs/2012.07436) and most recently, [Autoformer](https://arxiv.org/abs/2106.13008). In the following sections, we will deep dive into the Autoformer and outline how it can be implemented in PyTorch. 

# Autoformer

## Overview

Autoformer is a tranformer architecture for LSTF that aims to surmount the quadratic complexity of the attention mechanism and enhance prediction accuracy. In doing so, the Autoformer proposes two novel components, the Series Deocmposition Block and the Autocorrelation Block, that function as follows:

- **Series Decomposition Block:** Block that decomposes series into seasonal and trend-cyclical components. Used throughout the architecture to allow for the progressive decomposition of complex time series. 
- **Autocorrelation Block:** Self attention mechanism that conducts dependecy discovery and representation aggregation at the subseries level. Assuming that sub-series at the same phase position exhibit similar temporal processes, the Autocorrelation block construct series level connections based on the process similarity which is derived using series periodicity. 

At the top level, autoformer consists of an encoder and a decoder, each of which is composed sets of Auto-Correlation, Series Decomposition and Feed Forward Blocks with residual connections. 

<p align="center">
<img width="800" alt="Screen Shot 2021-09-28 at 5 41 20 PM" src="https://user-images.githubusercontent.com/34798787/149202576-720856f7-c827-4e3d-85d7-0053328e1c8d.png">  
</p>

With a high level understanding of the architecture in place, the following sections will outline the Series Deomposition Block and Autocorrelation block in detail. 

## Series Decomposition Block


### Overview

Series decomposition is often used in time series analysis to deconstruct a time series into a set of components. Each component represents an underlying category of pattern that is more predictable. In an attempt to leverage this inherent temporal structure, some recent approaches to time series forecasting ([NeuralProphet](https://arxiv.org/abs/2111.15397?fbclid=IwAR2vCkHYiy5yuPPjWXpJgAJs-uD5NkH4liORt1ch4a6X_kmpMqagGtXyez4), [NBEATS](https://arxiv.org/abs/1905.10437) and [DeepGLO](https://arxiv.org/abs/1905.03806)) employ decomposition. However, its use is limited to a pre-processing step applied to historical time series prior to predicting future time series. This overlooks the hierarchical interaction between underlying patterns of series in the long term future.

Autoformer proposes the Series Decomposition Block as an inner block for deep models. It aims to decompose time series into seasonal and trend-cyclical parts which reflect the long-term progression and seasonality of the series, respectively. As an inner block, The block the hidden series is progressively decomposed through the forecasting process including both past series and predicted intermediate results. Given an input sequence $X$, the seasonal component $X_{s}$ and trend-cyclical component $X_{t}$ are computed as follows:

<p align="center">
$X_{t} = AvgPool(Padding(X))$ 
<br>
<br>
$X_{s} = X - X_{t}$
</p>

To summarize the Series Decomposition Operation we use:
<p align="center">
$X_{s}, X_{t} = SeriesDecomp(X)$
</p>

### Implementation

The Series Decomposition Block simply computes $X_{t}$ as a rolling moving average over the time series. In turn, $X_{s}$ is computed as the difference between the time series and $X_{t}$. This can be easily translated to PyTorch as follows:

```python
class SeriesDecomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(SeriesDecomp, self).__init__()
        self.moving_avg = MovingAverage(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean
```
where moving average is a custom-defined module that computes a moving average over the time series. 

## Autocorrelation Block

### Overview
In the vanilla self-attention mechanism, pointwise dependency and aggregation is used. As discussed, this presents an issue in the case of LSTF because of the quadratic memory and compute as a function of the sequemce length. Modifications to the self attention mechanism have been proposed to sparsify the attention matrix so that the compute and memory requirements are feasible in the case of long sequences. However, by sparsifying the attention matrix, this creates an information usage bottleneck that can lead to degradation in the prediction accuracy of the model. 

To overcome the quadratic compute and memory of attention while avoiding the information bottekneck presented by sparisified attention, the Autocorrelation Mechanism is proposed. The Autocorrelation mechanism subsititutes sparsified point-wise connections in favour of a series-wise connections. Auto-Correlation discovers the period-based dependencies by calculating the series autocorrelation and aggregates similar subseries by time delay aggregation. Below offers some insight into how the AutoCorrelation mechanism compares to other variants of attention that have been proposed. 


<p align="center">
<img width="800" alt="Screen Shot 2022-01-25 at 4 44 17 PM" src="https://user-images.githubusercontent.com/34798787/151065976-f965c6d9-9322-4f4a-8a17-41ca86ebe6a0.png">
</p>



### Period-Based Dependencies

**Overview**

It is observed that the same phase position among periods naturally
provides similar sub-processes. Thus, the autocorrelation $R_{XX}(\tau)$ of a series $X_{t}$ and its $\tau$ lag series $X_{t - \tau}$ is used as similarty measure. The lags $\tau_{1},...,\tau_{k}$ with the highest autocorrelation are used to derive the period-based dependencies and can be weighted be the corresponding autocorrelation. $R_{XX}(\tau)$ is calculated as follows: 

<p align="center">
$R_{XX}(\tau) = \lim_{L \to ∞} \sum_{t=1}^{L} X_{t} X_{t - \tau}$
</p>
<br>

**Efficient Computation** 
In its current form, $R_{XX}(\tau)$ is not easily computable. Drawing on a wealth of reasearch in signal processing, $R_{XX}(\tau)$ can be computed using the Fast Fourier Transform. The computation accross all possible lags $ \in {1,...,L}$ in a series can be computed in $O(LlogL)$ time. 

### Time Delay Aggregation
The period-based dependencies connect the sub-series among estimated
periods. Thus, the time delay aggregation block is used to roll the series based
on selected time delay $\tau_{1},...,\tau_{k}$. This operation can align similar sub-series that are at the same phase position of estimated periods, which is different from the point-wise dot-product aggregation
in self-attention family. Finally, we aggregate the sub-series by softmax normalized confidences. The time delay aggregation block can be visualized as follows:

<p align="center">
<img width="646" alt="Screen Shot 2022-01-25 at 5 40 28 PM" src="https://user-images.githubusercontent.com/34798787/151071776-67c39999-567e-49ac-a94f-639e586cfa67.png">
</p>

### Attention Computation

**Single Head** 

For the single head situation and time series $X$ with length-$L$, after the projector, we get query $Q$, key
$K$ and value $V$. Thus, it can replace self-attention seamlessly. The Auto-Correlation mechanism is:

<p align="center">
$\tau_{1},...,\tau_{k} = argTopK(R_{Q, K}(\tau)) $ 
<br>
<br>
$R_{Q, K}(\tau_{1}),...,R_{Q, K}(\tau_{K}) = SoftMax(R_{Q, K}(\tau_{1}),...,R_{Q, K}(\tau_{K}))$
<br>
<br>
$Auto-Correlation(Q, K, V) = \sum_{i=1}^{k} Roll(V, \tau_{i}) R_{Q, K}(\tau_{i})$
</p>
where $argTopk(·)$ gets the arguments of the Topk autocorrelations and let $k = c × log L$ where c is a hyper-parameter. $R_{Q, K}$ is autocorrelation between series $Q$ and $K$. $Roll(X , \tau)$ represents the
operation to $X$ with time delay $\tau$ , during which elements that are shifted beyond the first positionare re-introduced at the last position. For the encoder-decoder Auto-Correlation, $K, V$ are from the encoder $X_{en}^{N}$
and will be resized to length-$O$, $Q$ is from the previous block of the decoder.

**Multi-Head** 

For the multi-head version, with hidden variables of $d_{model}$ channels, $h$ heads, the query, key and value for the $i$-th head are $Q_{i}, K_{i}, V_{i} \in \mathbb{R}^{L x \frac{d_{model}}{h}}, i \in {1,...h} $ the process is: 
<p align="center">
$Multihead(Q, K, V) = W_{output} * Concat(head_{1},...,head_{h})$ 
<br>
<br>
$head_{i} = Auto-Correlation(Q_{i}, K_{i}, V_{i})$
</p>

An overview of the AutoCorrelation Mechanism for the Multihead case can be visualized as follows:
<p align="center">
<img width="430" alt="Screen Shot 2022-01-25 at 6 19 35 PM" src="https://user-images.githubusercontent.com/34798787/151076113-eef0199d-f2cd-4f38-9079-306fd331881b.png">
</p>

### Model Inputs

**Model Overview**

At the top level, autoformer consists of an encoder and a decoder, each of which is composed sets of Auto-Correlation, Series Decomposition and Feed Forward Blocks with residual connections. The Encoder eliminates the long-term trend-cyclical part by series decomposition blocks and focuses on seasonal pattern modelling. Alternatively, the Decoder accumulates trend part extracted from hidden variables progressively. The past seasonal information from the encoder is utilized by the encoder-decoder Auto-Correlation. 

<p align="center">
<img width="800" alt="Screen Shot 2021-09-28 at 5 41 20 PM" src="https://user-images.githubusercontent.com/34798787/149202576-720856f7-c827-4e3d-85d7-0053328e1c8d.png">  
</p>

**Encoder**

The input to the encoder is the last $I$ time steps $X_{en}  ϵ  $ R<sup>$I x d$</sup>. Alternatively, the input to the decoder consists of Seasonal Part $X_{des} ϵ $ R<sup>($\frac{I}{2} + O)x d$</sup> and Trend-Cyclical Part $X_{det} ϵ $ R<sup>($\frac{I}{2} + O)x d$</sup> to be refined. For the decoder, each initialization is composed of: the latter half of encoders input $X_{en}$ with length $\frac{I}{2}$ to provide recent information and placeholders with length O filled by scalars. The initialization can be formalized as following: 

<p align="center">
$X_{ens}, X_{ent} = SeriesDecomp(X_{en\frac{I}{2}:I})$
<br>
<br>
$X_{des} = Concat(X_{ens}, X_{0})$
<br>
<br>
$X_{det} = Concat(X_{ent}, X_{mean})$
</p>

**Decoder** 

The decoder contains two parts: the accumulation structure for trend-cyclical components and the stacked Auto-Correlation mechanism for seasonal components. Each decoder layer contains the inner Auto-Correlation and encoder-decoder Auto-Correlation, which can refine the prediction and utilize the past seasonal information respectively. Note that the model extracts
the potential trend from the intermediate hidden variables during the decoder, allowing Autoformer to progressively refine the trend prediction and eliminate interference information for period-based dependencies discovery in Auto-Correlation. Suppose there are $M$ decoder layers. With the latent variable $X_{en}^{N}$ from the encoder, the equaltions of the $l$-th decode rlayer can be summarized as: 

<p align="center">
$X_{de}^{l} = Decoder(X_{de}^{l-1}, X_{en}^{N})$
</p>
and consists of the following operations: 
<p align="center">
$S_{de}^{l, 1}, T_{de}^{l, 1} = SeriesDecomp(Auto-Correlation(X_{de}^{l-1}) + X_{de}^{l})$
<br> 
<br>
$S_{de}^{l, 2}, T_{de}^{l, 2} = SeriesDecomp(Auto-Correlation(S_{de}^{l, 1}, X_{en}^{N}) + S_{de}^{l, 1})$
<br>
<br>
$S_{de}^{l, 3}, T_{de}^{l, 3} = SeriesDecomp(FeedForward(S_{de}^{l, 2}) + S_{de}^{l, 2}$
<br>
<br>
$T_{de}^{l} = T_{de}^{l-1} + W_{l, 1} * T_{de}^{l, 1} + W_{l, 2} * T_{de}^{l, 2} + W_{l, 2} * _{de}^{l, 3}$
</p>
where $X_{de}^{l} = S_{de}^{l, 3}, l \in {1,...,M}$ denotes the output of the $l-th$ decodr layer. $X_{de}^{0}$ is embedded from $X_{des}$ for deep transform and $T_{de}^{0} = X_{det}$ is for accumulation. $S_{de}^{l, i}, T_{de}^{l, i}, i \in {1, 2, 3}$ represent the seasonal component and the trend-cyclical component after the $i$-th series decomposition block in the $l$-th layer respectively. $W_{l, i}, i \in {1, 2, 3}$ represents the prokector for the $i$-th extracted trend $T_{de}^{l, i}$.

The final prediction is the sum of the two refined decomposed components as follows: 

<p align="center">
$ \hat{y} = W_{S} * X_{de}^{M} + T_{de}^{M}$
</p>

where $W_{s}$ is used to project the deep transformed seasonal components $X_{de}^{M}$ to the target dimension.